In [2]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/CNN/cnn_ext_model/ext_model.ipynb'

In [3]:
%run {RUN_DIR}

In [5]:
class RnnBasicModel(CnnExtModel):
    pass

In [ ]:
def rnn_basic_alloc_rnn_layer(self, input_shape, hconfig):
    # 입력 출력의 시계열 인지 확인
    inseq = get_conf_param(hconfig, 'inseq', True)
    outseq = get_conf_param(hconfig, 'outseq', True)
    
    if inseq:
    # 시간대최대 길이timesteps1, 시간대별 입력 벡터 크기= timefeats
    # 시간축 크기를 나타내며 길이표시를 위해
    # 한칸더 할당되었음을 표시하기 위해 1을 붙임
    #timefeats -> x1~xn
    
        timesteps1, timefeats = input_shape
    else:
        timesteps1 = get_conf_param(hconfig, 'timesteps') + 1
        timefeats = np.prod(input_shape)
    # 순환백터의 크기     
    # hidden
    recur_size = get_conf_param(hconfig, 'recur_size')
    # 가중치와 편향파라미터 생성
    ex_inp_dim = timefeats + recur_size
    # 시간대별입력 + 순환벡터의 확장크기 => w_param
    # bias -> recur_size
#     weight, bias = self.alloc_param_pair([ex_inp_dim, recur_size])
    # timesteps 1 -> 나중에 forward or back 반복제어용 
    if outseq:
        # 시계열데이터일경우
        output_shape = [timesteps1, recur_size]
        
    else:
        # 시계열 데이터가 아닐경우 
        
        output_shape = [recur_size]
#     rnn_info = [inseq, outseq, timesteps1, timefeats, recur_size]
    params = nn.RNN(input_size=timefeats, hidden_size=recur_size,batch_first=False)
    rnn = RNN(params)
    return [rnn], output_shape
#     return {'w':weight, 'b': bias, 'info':rnn_info}, output_shape


RnnBasicModel.alloc_rnn_layer = rnn_basic_alloc_rnn_layer

In [ ]:
class RNN(nn.Module):
    def __init__(self,rnn):
        super().__init__()
        self.rnn = rnn
    def forward(self,x):    
        print(x.shape)
        x=x.transpose(0,1)
        output,hiddens=self.rnn(x)
        return output[-1]

In [ ]:
def eval_accuracy_torch(self, x, y, output=None):
    if output is None:
        with torch.no_grad():
            output = self.model.forward(x)

    accuracy = self.dataset.eval_accuracy(x, y, output)
    
    return accuracy

RnnBasicModel.eval_accuracy = eval_accuracy_torch

In [ ]:
def test_torch(self):
    with torch.no_grad():
#         print(self.model.parameters)
        test_X,test_y=self.dataset.get_test_data()
#         test_X=torch.from_numpy(test_X).float()
#         test_y=torch.from_numpy(test_y).float()
        time1 = int(time.time())        
        acc = self.eval_accuracy(test_X, test_y)
        time2 = int(time.time())
        self.dataset.test_prt_result(self.name, acc, time2-time1)

RnnBasicModel.test = test_torch

In [ ]:
def mlp_get_estimate_torch(self, x):
    with torch.no_grad():
#         x=x.view([-1,3,96,96])
#         x=torch.from_numpy(x).float()
        output = self.model.forward(x)
    estimate = self.dataset.get_estimate(output)
    return estimate

RnnBasicModel.get_estimate = mlp_get_estimate_torch